#  Retrain GPT-2 and Generate Title


In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
!pip install -q arxivscraper
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import pandas as pd
import numpy as np
import arxivscraper as ax

## GPU

Verify which GPU is active by running the cell below.

In [10]:
!nvidia-smi

Thu Mar 19 20:33:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Scrape data

In [0]:
'''
# scraper for arxiv physics
scraper = ax.Scraper(category='physics', date_from='2019-05-01',
                     date_until='2019-07-03', t=10,
                     filters={'categories':['quant-ph']})

# scraper for arxiv q-bio
scraper = ax.Scraper(category='q-bio', date_from='2016-08-01',
                     date_until='2019-07-01', t=10, 
                     filters={'categories':['q-bio.GN', 'q-bio.NC']})
'''

# scraper for arxiv stat.ml
scraper = ax.Scraper(category='stat', date_from='2019-08-01',
                     date_until='2020-03-01', t=10, 
                     filters={'categories':['stat.ml'],'abstract':['learning']})
output = scraper.scrape()


# cols = ('id', 'title', 'categories', 'abstract', 'doi', 'created', 'updated', 'authors')
titles = [' '.join(o['title'].split()) for o in output]
np.savetxt('titles.csv', np.array(titles), fmt='%s')

## Fine-Tune GPT-2

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [13]:
model_name = "124M"
gpt2.download_gpt2(model_name=model_name)   

Fetching checkpoint: 1.05Mit [00:00, 757Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 156Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 782Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:05, 93.8Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 607Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 209Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 310Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [14]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
file_name = "titles.csv"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [0]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              'titles.csv',
              model_name='124M',
              steps=200,
              run_name='run2',
              save_every=100,
              sample_every=25)   # steps is max number of training steps

gpt2.generate(sess)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 57.24it/s]

Loading dataset...


dataset has 301567 tokens
Training...
[1 | 8.35] loss=2.33 avg=2.33
[2 | 9.24] loss=2.52 avg=2.42
[3 | 10.12] loss=2.36 avg=2.40
[4 | 11.01] loss=2.32 avg=2.38
[5 | 11.89] loss=2.16 avg=2.33
[6 | 12.78] loss=2.16 avg=2.30
[7 | 13.66] loss=2.38 avg=2.32
[8 | 14.54] loss=2.37 avg=2.32
[9 | 15.43] loss=2.40 avg=2.33
[10 | 16.31] loss=2.14 avg=2.31
[11 | 17.19] loss=2.02 avg=2.28
[12 | 18.08] loss=2.16 avg=2.27
[13 | 18.96] loss=2.17 avg=2.26
[14 | 19.84] loss=1.87 avg=2.23
[15 | 20.72] loss=2.23 avg=2.23
[16 | 21.60] loss=2.18 avg=2.23
[17 | 22.49] loss=2.05 avg=2.22
[18 | 23.37] loss=2.08 avg=2.21
[19 | 24.25] loss=2.07 avg=2.20
[20 | 25.13] loss=2.16 avg=2.20
[21 | 26.01] loss=2.10 avg=2.19
[22 | 26.90] loss=2.18 avg=2.19
[23 | 27.78] loss=2.03 avg=2.19
[24 | 28.67] loss=2.09 avg=2.18
[25 | 29.55] loss=2.03 avg=2.17
======== SAMPLE 1 ========
>startoftext|>a generalization with finite-state graphs for training neural networks: an example from the greek translation problems with nonzero 

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='run2')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name='run2')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [0]:
gpt2.generate(sess, run_name='run1')

FailedPreconditionError: ignored

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [0]:
phrase = """The rise of fake news has forced everyone with a social media account to become a detective, responsible for determining if a post is real before sharing it. Yet, fake news still slips past our defenses, proliferating on the web, amplified by ignorance and carelessness among real users. Beyond the spread of fear mongering and misinformation, fake news can do real damage [https://www.nbcnews.com/business/business-news/fake-news-can-cause-irreversible-damage-companies-sink-their-stock-n995436] to companies and individuals’ reputations. More than ever, we need better ways to detect fake news to prevent its spread. Some fake news is written by real people and is simply believable fiction, but the proliferation of fake news created with deep learning models can be generated in greater volumes and more rapidly than human texts, intensifying the problem. These natural language generation models have improved dramatically in recent years, making the task of catching them even more difficult, as they generate more believable text than ever. """

gpt2.generate(sess,
              length=100,
              temperature=0.7,
              prefix=phrase,
              nsamples=3,
              batch_size=3,
              )


The rise of fake news has forced everyone with a social media account to become a detective, responsible for determining if a post is real before sharing it. Yet, fake news still slips past our defenses, proliferating on the web, amplified by ignorance and carelessness among real users. Beyond the spread of fear mongering and misinformation, fake news can do real damage [https://www.nbcnews.com/business/business-news/fake-news-can-cause-irreversible-damage-companies-sink-their-stock-n995436] to companies and individuals’ reputations. More than ever, we need better ways to detect fake news to prevent its spread. Some fake news is written by real people and is simply believable fiction, but the proliferation of fake news created with deep learning models can be generated in greater volumes and more rapidly than human texts, intensifying the problem. These natural language generation models have improved dramatically in recent years, making the task of catching them even more difficult, a

In [0]:
phrase = """The rise of fake news has forced everyone with a social media account to become a detective, responsible for determining if a post is real before sharing it. Yet, fake news still slips past our defenses, proliferating on the web, amplified by ignorance and carelessness among real users. Beyond the spread of fear mongering and misinformation, fake news can do real damage to companies and individuals’ reputations. More than ever, we need better ways to detect fake news to prevent its spread. Some fake news is written by real people and is simply believable fiction, but the proliferation of fake news created with deep learning models can be generated in greater volumes and more rapidly than human texts, intensifying the problem. These natural language generation models have improved dramatically in recent years, making the task of catching them even more difficult, as they generate more believable text than ever. """
 # None is default
text = gpt2.generate(sess,
              length=40,
              temperature=0.7,
              prefix=phrase,
              nsamples=1,
              batch_size=1,
              return_as_list=True
             )


t = text[0].title()
t = t.replace('<|Startoftext|>', '').replace('\n', '') # remove extraneous stuff
t = t[:t.index('<|Endoftext|>')] # only get one title
print(t)

The Rise Of Fake News Has Forced Everyone With A Social Media Account To Become A Detective, Responsible For Determining If A Post Is Real Before Sharing It. Yet, Fake News Still Slips Past Our Defenses, Proliferating On The Web, Amplified By Ignorance And Carelessness Among Real Users. Beyond The Spread Of Fear Mongering And Misinformation, Fake News Can Do Real Damage To Companies And Individuals’ Reputations. More Than Ever, We Need Better Ways To Detect Fake News To Prevent Its Spread. Some Fake News Is Written By Real People And Is Simply Believable Fiction, But The Proliferation Of Fake News Created With Deep Learning Models Can Be Generated In Greater Volumes And More Rapidly Than Human Texts, Intensifying The Problem. These Natural Language Generation Models Have Improved Dramatically In Recent Years, Making The Task Of Catching Them Even More Difficult, As They Generate More Believable Text Than Ever. _________________________________Deep Learning For Automated Icd Coding: A R

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

new_phrase = """The rise of fake news has forced everyone with a social media account to become a detective, responsible for determining if a post is real before sharing it."""
gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=40,
                      temperature=0.7,
                      nsamples=100,
                      prefix=new_phrase,
                      batch_size=20
                      )

In [0]:
# may have to run twice to get file to download
files.download(gen_file)

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

Credit to Max Woolf: 

by [Max Woolf](http://minimaxir.com)

*Last updated: November 10th, 2019*


For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read Max Woolf's [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information.


# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.